In [2]:
import numpy as np
import matplotlib.pyplot as plt
import RBMF as rbef

from IPython.utils.path import ensure_dir_exists
fdir = 'figs/'
ensure_dir_exists(fdir)
resdir = 'res/'
ensure_dir_exists(resdir)

In [3]:
#savefig = True
savefig = False
if savefig:
    figsize = (7,4)
else:
    figsize = (14,8)

## by ODE with Runge-Kutta45

In [4]:
def deriv(t, Y, ny, iota, phi, delta):
    S, I, R, D = np.split(Y, np.cumsum(ny)[:-1])
    dYdt = np.zeros(np.sum(ny))
    
    dSdt  = - S*I@iota
    dI0dt =   S*I@iota - (phi[0]  + delta[0]) *I[0]
    dImdt = phi[:-1]*I[:-1] - (phi[1:] + delta[1:])*I[1:]
    dRdt  = phi[-1]*I[-1]
    dDdt  = delta@I
    return np.hstack((dSdt, dI0dt, dImdt, dRdt, dDdt))

## by SSA with Gillespie Algorithm

In [6]:
def updateStates(rExc, S, I, R, D):
    #print(rExc)
    nm = len(I)
    if rExc == 0:    # Infection
        S      -= 1
        I[0]   += 1
        return np.array(S), np.array(I), np.array(R), np.array(D)
    
    r = rExc - 1
    if r <= nm - 2:  # Changing Infectivity Level
        I[r]   -= 1
        I[r+1] += 1
        return np.array(S), np.array(I), np.array(R), np.array(D)
    if r == nm - 1:  # Changing Infectivity Level to Recovered
        I[r]   -= 1
        R      += 1
        return np.array(S), np.array(I), np.array(R), np.array(D)
          
    r -= nm    
    if r < nm:       # Death
        I[r]   -= 1
        D      += 1
        return np.array(S), np.array(I), np.array(R), np.array(D)   
    
def updatePropensities(S, I, R, D, N, iota, phi, delta):
    nm = len(I)
    kappa = np.zeros(2*nm+1)
    
    kappa[0]      = S * I @ iota / N  # Infection
    kappa[1:nm+1] =     I * phi       # Changing Infectivity Level / Recovery
    kappa[nm+1: ] =     I * delta     # Death
    return kappa


### Parameters:

In [ ]:
T     = 300
N     = 1000
Ii0   = 3

Rinit    = np.array([0])
Dinit    = np.array([0])

s = 0

iota0  = 0.2
rho0   = 0.02
delta0 = 0.01

infMean = 0.4